In [89]:
with open("SeinUndZeit.txt",'r',encoding='utf-8') as f:
    text = f.read()

    


In [90]:
print("Length of dataset in characters : ", len(text))

Length of dataset in characters :  1140505


In [91]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"$%&'()*,-./0123456789:;<=>?ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_abcdefghijklmnopqrstuvwxyz{|}~£§©«®°»ÄÖÜßäöü‘’“”„•■
113


In [92]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda j: ''.join([itos[c] for c in j])

In [93]:
# encode the entire text and store it into a torch.Tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
#print(data[:100])

torch.Size([1140505]) torch.int64


In [94]:
# split up the data into train and validation sets
n =  int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]



In [95]:
block_size = 8
train_data[:block_size+1]

tensor([49, 35, 39, 44,  1, 51, 44, 34,  1])

In [96]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    predictors = x[:t+1]
    target = y[t]
    print(f"input : {predictors} target : {target}")




input : tensor([49]) target : 35
input : tensor([49, 35]) target : 39
input : tensor([49, 35, 39]) target : 44
input : tensor([49, 35, 39, 44]) target : 1
input : tensor([49, 35, 39, 44,  1]) target : 51
input : tensor([49, 35, 39, 44,  1, 51]) target : 44
input : tensor([49, 35, 39, 44,  1, 51, 44]) target : 34
input : tensor([49, 35, 39, 44,  1, 51, 44, 34]) target : 1


In [97]:
torch.manual_seed(100)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context size for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y
xb, yb = get_batch('train')
print('inputs : ')
print(xb.shape)
print(xb)
print('targets :')
print(yb.shape)
print(yb)


for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target : {target}")




inputs : 
torch.Size([4, 8])
tensor([[104,  75,  75,  66,  75,   1,  98,  63],
        [ 80,  80,  66,  75,   1,  53,  66,  70],
        [ 64,  69,  81,  66,  81,  66,   1,  82],
        [  1,  64,  76,  68,  70,  81,  62,  75]])
targets :
torch.Size([4, 8])
tensor([[75, 75, 66, 75,  1, 98, 63, 70],
        [80, 66, 75,  1, 53, 66, 70, 80],
        [69, 81, 66, 81, 66,  1, 82, 75],
        [64, 76, 68, 70, 81, 62, 75, 80]])
when input is [104] the target : 75
when input is [104, 75] the target : 75
when input is [104, 75, 75] the target : 66
when input is [104, 75, 75, 66] the target : 75
when input is [104, 75, 75, 66, 75] the target : 1
when input is [104, 75, 75, 66, 75, 1] the target : 98
when input is [104, 75, 75, 66, 75, 1, 98] the target : 63
when input is [104, 75, 75, 66, 75, 1, 98, 63] the target : 70
when input is [80] the target : 80
when input is [80, 80] the target : 66
when input is [80, 80, 66] the target : 75
when input is [80, 80, 66, 75] the target : 1
when input is

In [98]:
import torch
import torch.nn as nn 
from torch.nn import functional as F 
torch.manual_seed(1337)

class BigrammLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx) #(B, T, C) 
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx


n = BigrammLanguageModel(vocab_size)
logits, loss = n(xb,yb)
print(logits.shape)
print(loss)

torch.Size([32, 113])
tensor(5.2331, grad_fn=<NllLossBackward0>)


In [99]:
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(n.generate(idx, max_new_tokens = 400)[0].tolist()))



X”/ßYo^«n=§QÄ®^«j{8H“LZ85HAr2
R7ä®vyLnsw.JjGK•iuLZSI=L(s_?,cgU1ß■b:^rXNIUf§!N‘PÖ8p®£w*;Lj2j<zhHßggtl5£VC21.P6DXNo*\R 5M&lCyi‘ä[I?x~0
&Pögdu(ICE0;^o6(l_eÜtn
s[ 4«8CÖ$uv 2Jj*{9cS\ä%ÖV”„L‘9H"Ä®';{}(QZGj,e'J  1”bQFä^“Hr©|cÜ)q7UL"u='V~q"uSr.c5lGGmW}&„{0jd*m,8Oa0_AS!$Y4wZYxQ1t°o®U^£_~q.0%$«4z■vy<,/XpWJ{’u©%cD«stVM=AT"%&u“4ö9(;Lö?b£’uR£GFi»„0.w.°vV•Ü©s>^“a|86ö}rz•6fEIJ/3=L(Y T•cfp>-WüoP3)“eZ{’op|SyJHgHxj


In [100]:
# create a Pytorch optimizer
optimizer = torch.optim.Adam(n.parameters(),lr=1e-3)

In [101]:
batch_size  = 32 
for steps in range(10000):
    # sample a batch of data
    xb , yb = get_batch('train')

    # evaluate the lass
    logits, loss = n(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())



2.213249444961548


In [102]:
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(n.generate(idx, max_new_tokens = 400)[0].tolist()))



dies damandielikte wet i=0, dageichensvDan« d zunheichkle gas jvonodileit: Spheiem deschltzusss imes eit, dalun br« s eneseisetheitzhit, des aglt dins Wichafiht- Wis F„>« Albeie ureias den. 
iertigersier welst. inogendespt Dungast »L(4Cs or iköhunh u 

gegloch Ses Ttl. Esos in d 
wer der aum vehün dieiondeno Undast derdasieihei- ragichalim Das- ['zuielirs 
s dalie 
Abg K\ Gewenaun 

enseimiexieru
